In [2]:
import xarray as xr
import numpy as np


time = np.arange(0, 3600, 1)  # seconds
range_bin = np.arange(0, 1800, 1)  # samples

ds = xr.Dataset(
    data_vars=dict(
        Sv=(["time", "range_bin"], np.random.randn(time.size, range_bin.size).astype("float32")),
    ),
    coords=dict(time=("time", time), range_bin=("range_bin", range_bin)),
    attrs={"convention": "SONAR-netCDF4 v1.0"},
)

ds

<xarray.Dataset> Size: 26MB
Dimensions:    (time: 3600, range_bin: 1800)
Coordinates:
  * time       (time) int64 29kB 0 1 2 3 4 5 6 ... 3594 3595 3596 3597 3598 3599
  * range_bin  (range_bin) int64 14kB 0 1 2 3 4 5 ... 1795 1796 1797 1798 1799
Data variables:
    Sv         (time, range_bin) float32 26MB -0.2556 -0.04937 ... 0.8653 -1.686
Attributes:
    convention:  SONAR-netCDF4 v1.0

In [3]:
ds_chunked = ds.chunk({"time": 300, "range_bin": 256})
ds_chunked

<xarray.Dataset> Size: 26MB
Dimensions:    (time: 3600, range_bin: 1800)
Coordinates:
  * time       (time) int64 29kB 0 1 2 3 4 5 6 ... 3594 3595 3596 3597 3598 3599
  * range_bin  (range_bin) int64 14kB 0 1 2 3 4 5 ... 1795 1796 1797 1798 1799
Data variables:
    Sv         (time, range_bin) float32 26MB dask.array<chunksize=(300, 256), meta=np.ndarray>
Attributes:
    convention:  SONAR-netCDF4 v1.0

In [4]:
ds_chunked.to_zarr("s3://xarray-zarr-demo/sonar_scan.zarr",
           mode="w",
           consolidated=True,
           zarr_format=2)

In [9]:
ds2 = xr.open_zarr("s3://xarray-zarr-demo/sonar_scan.zarr")
# index-based window
subset = ds2["Sv"].isel(time=slice(600, 1200), range_bin=slice(0, 512)).load()
subset

<xarray.DataArray 'Sv' (time: 600, range_bin: 512)> Size: 1MB
array([[-1.9821801 , -0.1183504 ,  0.6789474 , ..., -0.17891653,
         1.48781   , -1.9366693 ],
       [-0.44543803,  0.12147106, -0.09992775, ..., -0.44663563,
         0.65634173,  0.7648874 ],
       [ 0.51180357, -1.6231524 , -0.87909514, ..., -0.22886525,
         1.4416335 ,  0.17588517],
       ...,
       [ 0.37431914, -0.278583  ,  0.08444498, ..., -0.37321782,
         0.27959663,  0.01239446],
       [ 0.29799816,  0.51334345,  0.1973742 , ...,  0.63495725,
        -0.07416622, -1.2060444 ],
       [-0.4247977 ,  1.069645  , -0.7454623 , ..., -0.6789483 ,
        -1.1198004 ,  0.15608841]], shape=(600, 512), dtype=float32)
Coordinates:
  * range_bin  (range_bin) int64 4kB 0 1 2 3 4 5 6 ... 506 507 508 509 510 511
  * time       (time) int64 5kB 600 601 602 603 604 ... 1195 1196 1197 1198 1199

In [3]:
from dask.distributed import LocalCluster

cluster = LocalCluster()          # Fully-featured local Dask cluster
client = cluster.get_client()

client

/Users/andrei/oceanstream/temp-calculations/venv/lib/python3.11/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 57540 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:57540/status,
Dashboard: http://127.0.0.1:57540/status,Workers: 5
Total threads: 10,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:57541,Workers: 0
Dashboard: http://127.0.0.1:57540/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:57557,Total threads: 2
Dashboard: http://127.0.0.1:57563/status,Memory: 6.40 GiB
Nanny: tcp://127.0.0.1:57544,
